# Readme

seria verzii 4 ma byt kompletne prepracovana a zjednodusena verzia skriptu bez zbytocneho balastu. Vychadza z verzie 3.10 ( https://colab.research.google.com/drive/1umwQjAAf---S-Tjk-s5ewnAE4dCHBVao?usp=sharing )

## Versions and updates

**v3.9**: https://colab.research.google.com/drive/1pzCfFINtVYPA3NoawgcCtrwRvIpomY_z?usp=sharing v tomto scripte boli doplnene chybajuce ID_HP_ZP vo formate "uzsX" tak, ako mi to Peto povedal a mam napisane rucne v poznamkach zo stretka. V casti, kde concatujem uzs_jzs a hp namiesto jednotlivych uzs_jzs filov vlozim upraveny uzs_jzs_allhic.csv

**v3.10** upravene MS attribution (enngine) podla poslednych Petovych rad - filtracia bola zla, mam hladat co najnizsi subor aj co najnizsi "prior" (co je vraj povodne nejake cislo riadku alebo tak)

**v4.0:** Vsimli sme si, ze niektore ID_HP_ZP boli duplikatne 2x. Problem bol v subore "obce_gps_simple.xlsx" kde napajam obce a GPS pacientov, ze niektore kody obci boli 2x s roznymi nazvami, napr. Povazska Bystrica a Orlove. Po odstaraneni duplicitnych riadkov v tomto exceli mam uz opat okolo 1 mil. a 8 tis. riadkov. Unique ID_HP_ZP je asi o 3 tis. menej - na to sa pozriem nabuduce

**v4.1**: Vzhladom na to, ze vo verzii 4.0 boli stale nejake duplikaty (asi o 3 tis. menej unique ID_HP_ZP ako pocet celkovych riadkov), po zobrazeni tychto duplikatov sa javilo, ze ide v zasade o tych istych pacientov. Rozdiel som si vsimol asi len v bydlisku. Neviem to na 100% vysvetlit, ale moze sa jednat len o nejaky sum, zle zadane data a pod. **Rozhodol som sa vsetky duplikaty odstranit a nechat kazde ID_HP_ZP iba unikatne raz**

Dalej bol pridany Special Output for PowerBI purposes, kedze vizualizacia dat v PowerBI si vyzaduje trochu inu agregaciu a robilo sa mi to lepsie v Pythone ako priamo v PowerBI. Pri tom zgrupeni podla okresu pacienta a kodu PZS mi to par tisic riadkov zmizlo, neviem zatial, preco.

**v4.2**: Special Output for PowerBI dalej detailizovany tak, aby som si vedel na mape zobrazit pre nemocnicu, z akeho okresu su pacienti, ale aj filtrovat podla nazvu programu a urovne medicinskej sluzby.

PROBLEM 1: pri takomto zobrazeni dat mi z doposial nezisteneho dovodu par tisic pacientov zmizne. Celkovo som mal okolo 1,005 tis. unique ID_HP_ZP, cize pacientov. Pri tomto zgrupeni len po okresoch ich o par tisic zmizne, a pri dalsom zgrupeni po okresoch, ms a programoch este o par tisic zmizne.

PROBLEM 2: priradovanie MS k programom a urovne> kedze mame primarne a zdielane MS a niektore MS maju rozne urovne (napr. pre deti), jednemu kodu MS je problem priradit jednoznacnu uroven. Ked som tam najskor dal komplet excel zoznam MS pre rok 2022, vzniklo mi vela duplikatov. Takze pre kazdy kod MS som potreboval len 1 riadok s 1 programom a urovnou. V exceli, z ktoreho cerpam, som si taketo priradenie zjednodusil, avsak moze vytvarat nepresnosti.

PROBLEM 3/ SOLVED (asi): Kedze pri niektorych okresoch, kde som nevedel kod ZUI jednoznacne priradit k okresu alebo bola nejaka pochybnost, som do excelu s GPS obcami, z ktoreho cerpam, najskor taketo okresy daval s ? a trochu inymi GPS koordinatmi. Cize som mal napriklad ? Povazska Bystrica. To na mape ale nevyzeralo dobre, tak som vsade pripradil jednoznacny okres s rovnakymi koordinatmi. Nejaky sum v tych datach ale moze nastat. Naviac som bratislavske a kosicke okresy zgrupil do jedneho, nech to na mape lepsie vyzera (BA bolo napr. I az V tusim).

**v4.3**: kvoli vystupu pre PowerBI a naslednu tvorbu mapy som musel zmenit subor "obce_gps_simple" na diakritiku, inak mi to nenaparovalo na mapu od Michala (json format mapy s okresmi SR do PowerBI)

Pridany este detailnejsi prehlad "patient_distribution_detailed_withMS_sorted" pre mSMA - aby bolo vidiet aj konkretne medicinske sluzby

# Data preparation

## Imports

Colab cooperation

In [1]:
#from google.colab import drive
#drive.mount('/content/drive/')

Mounted at /content/drive/


Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import json
import re

import seaborn as sns
import os

## Datasets imports

Data s ktorymi pracujem Peto popisal tuto: https://healthgovsk-my.sharepoint.com/:w:/g/personal/juraj_hunak_health_gov_sk/EQxs5uqgMl1HmnpYdPQ2pMYBmtI4itr_ylY-AosrChZq7g?e=4k8fS5

### Hospitalizations

•	OSN - General\04_Siet nemocnic\02_Vyhodnotenie_siete\OSN_MedicalServiceGrouping\Kroky_zaloha\uzs_ms_list.csv

In [3]:
filepath_tab1 = "C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\dataset.txt"
hospitalizations = pd.read_csv(filepath_tab1, dtype=str, delimiter="|")

In [4]:
#hospitalizations

In [5]:
hospitalizations.dtypes

id_hosp_nem      object
vek              object
HMOTNOST         object
UPV              object
typ_zs           object
vykon_lab        object
vykon            object
kod_ms           object
ms_name          object
drg              object
icd              object
icd_vedlajsia    object
icd_any          object
joiner           object
subor            object
prior            object
tazkosti         object
sig_op           object
vek_skupina      object
kod_nem          object
ID_HP_ZP         object
dtype: object

In [6]:
hospitalizations.shape

(3644380, 21)

### UZS_JZS files of all HICs

•	OSN - General\04_Siet nemocnic\02_Vyhodnotenie_siete\OSN_MedicalServiceGrouping\Data_ZP\2022\Opravene_data\xx\2022_xx_01_UZS_JZS.csv, kde xx je cislo poistovne

In [4]:
uzs_jzs_24 = pd.read_csv("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\2022_24_01_UZS_JZS.csv", delimiter='|', low_memory=False, dtype=str)
uzs_jzs_25 = pd.read_csv("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\2022_25_01_UZS_JZS.csv", delimiter='|', low_memory=False, dtype=str)
uzs_jzs_27 = pd.read_csv("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\2022_27_01_UZS_JZS.csv", delimiter='|', low_memory=False, dtype=str)

In [5]:
hp_24 = pd.read_csv("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\2022_24_02_HP.csv", delimiter='|', low_memory=False, dtype=str)
hp_25 = pd.read_csv("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\2022_25_02_HP.csv", delimiter='|', low_memory=False, dtype=str)
hp_27 = pd.read_csv("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\2022_27_02_HP.csv", delimiter='|', low_memory=False, dtype=str)

### Insurees

•	OSN - General\04_Siet nemocnic\02_Vyhodnotenie_siete\OSN_MedicalServiceGrouping\Data_ZP\2022\Opravene_data\xx\2022_xx_09_POISTENCI.csv, kde xx je cislo poistovne

In [6]:
insurees24 = pd.read_csv("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\2022_24_09_POISTENCI.csv", delimiter='|', low_memory=False, dtype=str)

In [10]:
#insurees24.head()

In [7]:
insurees25 = pd.read_csv("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\2022_25_09_POISTENCI.csv", delimiter='|', encoding='ISO-8859-1', low_memory=False, dtype=str)
# bol UnicodeDecodeError, preto ten zvlastny encoding

In [8]:
insurees27 = pd.read_csv("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\2022_27_09_POISTENCI.csv", delimiter='|', low_memory=False, dtype=str)

In [9]:
insurees_allhic = pd.concat([insurees24, insurees25, insurees27], ignore_index=True)

In [10]:
insurees_allhic.shape

(5540524, 11)

### PZS GPS

In [11]:
pzs_gps = pd.read_excel("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\PZS_zakl_info_GPS_bezprepojeni.xlsx", dtype=str)

In [16]:
#pzs_gps

In [12]:
pzs_gps.rename(columns={'IDENTIFZAR': 'kod_nem', 'SIDZAR_KOD_OBEC': 'KOD_OBEC_PZS', 'SIDZAR_KOD_OKRES_PZS': 'KOD_OKRES_PZS', 'lat_pzs_ifexists': 'lat_pzs', 'lng_pzs_ifexists': 'lng_pzs', 'NAZZAR': 'NAZOV_PZS'}, inplace=True)

### Patient residencies GPS

In [13]:
pacient_gps = pd.read_excel("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\obce_gps_simple.xlsx", dtype=str)

In [19]:
pacient_gps

,RESIDENCY,Obec_pacient_nazov,id_okres_pacient,Okres_pacient_nazov,okresne_mesto_zuj_pacient,okresne_mesto_pacient_nazov,lat_okres_pacient,lng_okres_pacient,residency_count
0,501280,Nesvady,401,Komárno,501026,Komárno,47.7657985805,18.1195514564,1
1,504025,Sala,405,Šaľa,504025,Šaľa,48.1567147391,17.8759711269,1
2,504092,Trnovec nad Vahom,405,Šaľa,504025,Šaľa,48.1567147391,17.8759711269,1
3,555509,Zalesie,108,Senec,508217,Senec,48.2198134892,17.3960613228,1
4,504131,Velke Ulany,202,Galanta,503665,Galanta,48.1899528911,17.7250271321,1
...,...,...,...,...,...,...,...,...,...
3280,NBG,BG,ZAHR,Zahranicie ine,ZAHR,Zahranicie ine,47.6877142798338,16.9126556922479,1
3281,UA2EC2,UA,UA,UA,UA,Ukrajina,48.5420509368223,22.3764011118678,1
3282,AT0142,AT,AT,AT,AT,Rakusko,48.1904354484935,16.4325044223195,1
3283,DE1276,DE,ZAHR,Zahranicie ine,ZAHR,Zahranicie ine,47.6877142798338,16.9126556922479,1


### MS levels

In [14]:
ms_levels = pd.read_excel("C:\\Users\\hunakj\\health.gov.sk\\IZA - Cestovanie za ZS\\2022\\data_inputs_other\\data\\uroven_MS_2022_iba_hlavne_programy.xlsx", dtype=str)

In [15]:
ms_levels.head()

,kod_ms,hlavny_program_cislo,hlavny_program_nazov,uroven_dospeli,uroven_deti,ms_uroven,ms_uroven_comparison,extract_main_program,number_main_program,comparisons_program_numbers,main_or_shared_program
0,01-01,1,Program urgentnej medicíny,IV,IV,IV,True,1,1,0,main_program
1,01-02,1,Program urgentnej medicíny,III,III,III,True,1,1,0,main_program
2,01-03,1,Program urgentnej medicíny,II,II,II,True,1,1,0,main_program
3,01-04,1,Program urgentnej medicíny,I,I,I,True,1,1,0,main_program
4,02-01,2,Program pre perioperačnú medicínu,V,NaN,V,EMPTY,2,2,0,main_program


##MS attribution

toto by mal byt engine, kde si hospitalizacie (jedna je na viacerych riadkoch) vytriedim, aby mi idealne na konci ostal jeden riadok s jednou relevantnou MS. Do budcna idealne aj s info o oddeleni a LOSe pacienta. Nateraz davam zjednodusene

In [22]:
hospitalizations_sorted = hospitalizations.sort_values(by=['subor', 'prior'], ascending=[True, True])
hospitalizations_filtered = hospitalizations_sorted.drop_duplicates(subset='ID_HP_ZP', keep='first')

In [23]:
hospitalizations_filtered.shape

(1005472, 21)

In [24]:
hospitalizations_filtered['ID_HP_ZP'].isna().sum()

0

## Merging all dataframes to get BIC_POI

### Concat and process UZS_JZS

update od verzie 3.9.: https://colab.research.google.com/drive/1pzCfFINtVYPA3NoawgcCtrwRvIpomY_z?usp=sharing v tomto scripte boli doplnene chybajuce ID_HP_ZP vo formate "uzsX" tak, ako mi to Peto povedal a mam napisane rucne v poznamkach zo stretka. Od verzie 4 to chcem mat vsetko spolu tu a nebudem mat "bocny skript". To len pre info

In [25]:
uzs_jzs_allhic = pd.concat([uzs_jzs_24, uzs_jzs_25, uzs_jzs_27], ignore_index=True)

In [26]:
# Filter the dataframe to keep rows where 'HOSP_TYP' is not equal to 'Z' - toto je podla Misky dolezite, lebo su to nejake pripocitatelne polozky alebo co
uzs_jzs_allhic = uzs_jzs_allhic[uzs_jzs_allhic['HOSP_TYP'] != 'Z']

In [27]:
uzs_jzs_allhic.shape

(1227522, 18)

#### Doplnanie chybajucich "uzsX" na miesto prazdnych ID_HP_ZP

dolezite, aby som potom mal cez co joinovat. Tam mi to Peto hovoril nejak tak, ze kedze Python indexuje od 0, mam davat vzdy plus 1. Lebo u neho bol vymysleny kod "uzs" + cislo riadka

In [28]:
for idx, value in enumerate(uzs_jzs_allhic['ID_HP_ZP']):
    if pd.isna(value):
        uzs_jzs_allhic.at[idx, 'ID_HP_ZP'] = "uzs" + str(idx + 1)

In [29]:
#uzs_jzs_allhic

### Concat uzs_jzs_allhic and hp dataframes into one SINGLE dataframe

In [30]:
dataframes = [uzs_jzs_allhic, hp_24, hp_25, hp_27]

uzs_jzs_hp_allhic = pd.concat(dataframes, ignore_index=True)

In [31]:
uzs_jzs_hp_allhic.shape

(1974961, 40)

In [32]:
#uzs_jzs_hp_allhic.dtypes

In [33]:
#uzs_jzs_hp_allhic

In [34]:
uzs_jzs_hp_allhic['ID_HP_ZP'].nunique()

1110768

In [35]:
uzs_jzs_hp_allhic['BIC_POI'].isna().sum()

6

In [36]:
uzs_jzs_hp_allhic['ID_HP_ZP'].isna().sum()

26646

### Merge uzs_jzs_hp_all with hospitalizations

In [37]:
merged_hospitalizations = hospitalizations_filtered.merge(uzs_jzs_hp_allhic[['ID_HP_ZP', 'BIC_POI']], on='ID_HP_ZP', how='left')

In [38]:
merged_hospitalizations.shape

(1816929, 22)

z nejakeho dovodu mi pocet oproti starsim skriptom narastol, vo verzii 3.5 mam napr 1,2 mil. riadkov, v 3.10 uz tiez 1,8 mil.

In [39]:
merged_hospitalizations['ID_HP_ZP'].nunique()

1005472

In [40]:
merged_hospitalizations['BIC_POI'].isna().sum()

982

toto dole sluzi na zobrazenie duplikatov, potreboval som sa namatkovo presvedcit, ze riadky pre 1 ID_HP_ZP su zdvojene, teda niektore. A diagnoza, kod_ms a dalsie su rovnake

In [41]:
# Find duplicate rows based on the 'ID_HP_ZP' column
duplicates = merged_hospitalizations[merged_hospitalizations.duplicated(subset='ID_HP_ZP', keep=False)]

In [42]:
#duplicates.head(500)

#### Keep each ID_HP_ZP just once

In [43]:
unique_merged_hospitalizations = merged_hospitalizations.drop_duplicates(subset='ID_HP_ZP', keep='first')

In [44]:
unique_merged_hospitalizations.shape

(1005472, 22)

In [45]:
unique_merged_hospitalizations['BIC_POI'].isna().sum()

982

## Merge "unique_merged_hospitalizations" with residencies

In [46]:
unique_merged_hospitalizations['ID_HP_ZP'] = unique_merged_hospitalizations['ID_HP_ZP'].apply(
    lambda x: str(int(float(x))) if re.match(r'^-?\d+(?:\.\d+)?$', x) else x
)

<ipython-input-46-f9afdf929a76>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_merged_hospitalizations['ID_HP_ZP'] = unique_merged_hospitalizations['ID_HP_ZP'].apply(


In [47]:
df_hospitalizations_residency = unique_merged_hospitalizations.merge(insurees_allhic[['BIC_POI', 'KOD_PRECHOD', 'KOD_TRVALY']], on='BIC_POI', how='left')

In [48]:
df_hospitalizations_residency.shape

(1008388, 24)

In [49]:
df_hospitalizations_residency.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907,NaN,529460
1,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222,NaN,510025
2,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8704994,NaN,506524
3,39507_22001949_P8948301,0,4000,24,DRG,8p133_20220316_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22001949,5016945,NaN,522279
4,39507_22002222_P8948301,0,1100,71,DRG,8p107_20220324_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22002222,2668229,NaN,522872


In [50]:
df_hospitalizations_residency['KOD_TRVALY'].isna().sum()

14408

tu mam zmenu oproti v3.10, tam som mal 12853 NaN hodnot, naopak dole, filtered_df mal shape az 948 tis. Myslim si, ze to moze byt tym, ako tu nacitavam automaticky vsetko ako stringy. Tam som robil rozne manipulacie a konvertovanie jednotlivych stlpcov, mozno sa Pandas pri tom sprava inak. Kusok vyssie tou komplikovanou lambdou som myslel, ze to nejak osetrim (lebo ze to asi robia tie BIC_POI kde mam nejaky zvlastny charakter), ale nijak to nepomohlo, datasety sa nezmenili

In [51]:
filtered_df = df_hospitalizations_residency[df_hospitalizations_residency['KOD_PRECHOD'].isin(['NaN', 'Nan', 'nan'])]

In [52]:
filtered_df.shape

(0, 24)

In [53]:
filtered_df = df_hospitalizations_residency[df_hospitalizations_residency['KOD_PRECHOD'].isna()]

In [54]:
filtered_df.shape

(961455, 24)

In [55]:
df_hospitalizations_residency['BIC_POI'].isna().sum()

982

### Residency - KOD_PRECHOD if existent otherwise KOD_TRVALY

In [56]:
df_hospitalizations_residency['RESIDENCY'] = df_hospitalizations_residency['KOD_PRECHOD']

df_hospitalizations_residency['RESIDENCY'] = np.where(
    df_hospitalizations_residency['KOD_PRECHOD'].isna(), # tuto cast kodu musim oproti v3.10 zmenit, preco je o par riadkov vyssie - filtered_df mam po isna()
    df_hospitalizations_residency['KOD_TRVALY'],
    df_hospitalizations_residency['KOD_PRECHOD']
)

In [57]:
df_hospitalizations_residency.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907,NaN,529460,529460
1,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222,NaN,510025,510025
2,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8704994,NaN,506524,506524
3,39507_22001949_P8948301,0,4000,24,DRG,8p133_20220316_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22001949,5016945,NaN,522279,522279
4,39507_22002222_P8948301,0,1100,71,DRG,8p107_20220324_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22002222,2668229,NaN,522872,522872


In [58]:
df_hospitalizations_residency['RESIDENCY'].isna().sum()

12935

mierne sa NaN pri residencies navysilo, neriesim (o par desiatok)

In [59]:
# vizualna kontrola, ci to funguje - zda sa, ze ano
filtered_df = df_hospitalizations_residency[~df_hospitalizations_residency['KOD_PRECHOD'].isna()]

# Display values in the other columns of the filtered DataFrame
filtered_df

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY
13,8873_22015652_P4305901,0,3000,358,DRG,8p133_20220923_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4305901,8873_22015652,8732094,505846,508217,505846
19,8873_22021515_P4305901,0,1310,217,DRG,8p133_20221212_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4305901,8873_22021515,2612911,509086,509086,509086
22,20223300007936_P3881101,0,1050,1080,DRG,8p133_20211215_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,20223300007936,788251001.0,509302,517402,509302
62,20226100042990_P7024901,0,3780,22,DRG,8p107_20220310_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P7024901,20226100042990,5717265001.0,508233,528960,508233
72,100000457593_P8948301,0,720,7,DRG,8p107_20220626_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,100000457593,100000075902,521299,521736,521299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008291,31064_22000311_P8985101,63,-2,-2,UH,NaN,.*,25-18,Bolesť na hrudníku,.*,z450,.*,.*,icd,19,62629,0,0,age19,P8985101,31064_22000311,7734468,522937,522279,522937
1008303,31064_22003209_P8985101,80,-2,-2,UH,NaN,.*,25-18,Bolesť na hrudníku,.*,z450,.*,.*,icd,19,62629,0,0,age19,P8985101,31064_22003209,2380959,599824,598224,599824
1008323,31064_22005941_P8985101,75,-2,-2,UH,NaN,.*,25-18,Bolesť na hrudníku,.*,z450,.*,.*,icd,19,62629,0,0,age19,P8985101,31064_22005941,4936810,522279,522741,522279
1008361,100000434715_P7794101,28,-1,-1,DRG,5l59153_20220309_P,.*,29-11,Dialýza,.*,z490,.*,.*,icd,19,62633,0,0,age19,P7794101,100000434715,100000912596,501263,501263,501263


## Pripajanie GPS PZS a dalsich info (nazov)

In [60]:
df_hospitalizations_residency = df_hospitalizations_residency.merge(pzs_gps[['kod_nem','NAZOV_PZS', 'KOD_OKRES_PZS', 'lat_pzs', 'lng_pzs']], on='kod_nem', how='left')

In [61]:
df_hospitalizations_residency['NAZOV_PZS'].isna().sum()

0

In [62]:
df_hospitalizations_residency[df_hospitalizations_residency['NAZOV_PZS'].isin(['NaN', 'Nan', 'nan', 'na', 'Na', ' ', ''])].shape

(0, 29)

In [63]:
# Get list of values from ColumnA where ColumnB has NaN values
list_of_values = df_hospitalizations_residency['kod_nem'][df_hospitalizations_residency['lat_pzs'].isna()].tolist()
unique_missing_pzs = df_hospitalizations_residency['kod_nem'][df_hospitalizations_residency['lat_pzs'].isna()].unique().tolist()

print(len(list_of_values))
print(df_hospitalizations_residency['kod_nem'][df_hospitalizations_residency['lat_pzs'].isna()].nunique())
print(unique_missing_pzs)

0
0
[]


## Priprajanie GPS okresu pacientov a dalsich info

In [64]:
df_hospitalizations_residency = df_hospitalizations_residency.merge(pacient_gps[['RESIDENCY','Obec_pacient_nazov', 'id_okres_pacient', 'Okres_pacient_nazov', 'lat_okres_pacient', 'lng_okres_pacient']], on='RESIDENCY', how='left')

In [65]:
df_hospitalizations_residency['id_okres_pacient'].isna().sum()

13056

zaujimave, oproti v3.10 mam dokonca menej asi o 50 NaN pri id_okres_pacient

Zoznam kodov obcii som manualne doplnal v exceli. Mal som tam par cudnych, ktorym som priradil obec podla toho, kde bola poskytnuta zdrav. star. I ked to nie je uplne najlepsi pristup, vid komentar z predchadzajucich verzii:

"update 12.12.2023: po tom, co som aktualizoval zoznam obci aj s chybajucimi kodmi*, tak mam uz len minimum "novych" chybajucich bydlisk pacientov. Gro je v zasade to, co uz bolo od zaciatku.

*kedze tie udaje (kody ZUJ) som nevedel nikde najst, kazdemu cudnemu ZUJ som
riradil okres podla toho, kde vyraznej casti pripadov bola poskytnuta zdrav. star. Na konci manualne pridane do suboru "obce_gps_simple.xlsx". Nie je to mozno uplne najspravnejsia metodika, lebo ak by pre nejake ZUJ napr. 50XXXX zo 120 osetrenych pacientov pripadalo 109 na okres Bratislava, este to neznamena, ze za tou ZS necestovali (a to chcem presne zistit). Ale je to vysoko pravdepodobne, ze ti pacienti su z BA a v tomto priklade tych 11 bratislavskych bolo osetrenych mimo BA"

## Pripajanie urovni MS, programov atd.

In [66]:
df_hospitalizations_residency = df_hospitalizations_residency.merge(ms_levels[['kod_ms','hlavny_program_cislo','hlavny_program_nazov','ms_uroven']], on='kod_ms', how='left')

In [67]:
list(df_hospitalizations_residency.columns)

['id_hosp_nem',
 'vek',
 'HMOTNOST',
 'UPV',
 'typ_zs',
 'vykon_lab',
 'vykon',
 'kod_ms',
 'ms_name',
 'drg',
 'icd',
 'icd_vedlajsia',
 'icd_any',
 'joiner',
 'subor',
 'prior',
 'tazkosti',
 'sig_op',
 'vek_skupina',
 'kod_nem',
 'ID_HP_ZP',
 'BIC_POI',
 'KOD_PRECHOD',
 'KOD_TRVALY',
 'RESIDENCY',
 'NAZOV_PZS',
 'KOD_OKRES_PZS',
 'lat_pzs',
 'lng_pzs',
 'Obec_pacient_nazov',
 'id_okres_pacient',
 'Okres_pacient_nazov',
 'lat_okres_pacient',
 'lng_okres_pacient',
 'hlavny_program_cislo',
 'hlavny_program_nazov',
 'ms_uroven']

In [68]:
df_hospitalizations_residency = df_hospitalizations_residency.reindex([
 'id_hosp_nem',
 'vek',
 'HMOTNOST',
 'UPV',
 'typ_zs',
 'vykon_lab',
 'vykon',
 'kod_ms',
 'ms_name',
 'ms_uroven',
 'hlavny_program_cislo',
 'hlavny_program_nazov',
 'drg',
 'icd',
 'icd_vedlajsia',
 'icd_any',
 'joiner',
 'subor',
 'prior',
 'tazkosti',
 'sig_op',
 'vek_skupina',
 'kod_nem',
 'ID_HP_ZP',
 'BIC_POI',
 'KOD_PRECHOD',
 'KOD_TRVALY',
 'RESIDENCY',
 'NAZOV_PZS',
 'KOD_OKRES_PZS',
 'lat_pzs',
 'lng_pzs',
 'Obec_pacient_nazov',
 'id_okres_pacient',
 'Okres_pacient_nazov',
 'lat_okres_pacient',
 'lng_okres_pacient',
],
                                                                      axis=1)

## Output

In [69]:
df_hospitalizations_residency.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,ms_uroven,hlavny_program_cislo,hlavny_program_nazov,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY,NAZOV_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs,Obec_pacient_nazov,id_okres_pacient,Okres_pacient_nazov,lat_okres_pacient,lng_okres_pacient
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",IV,43,Program intenzívnej starostlivosti v neonatológii,p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907,NaN,529460,529460,Univerzitná nemocnica Bratislava,102,48.156523,17.150507,Bratislava - Petrzalka,101,Bratislava I-V,48.1479920465,17.1009521666
1,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",IV,43,Program intenzívnej starostlivosti v neonatológii,p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222,NaN,510025,510025,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695,Rabca,507,Námestovo,49.403873208,19.4820418231
2,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",IV,43,Program intenzívnej starostlivosti v neonatológii,p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8704994,NaN,506524,506524,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695,Stara Tura,304,Nové Mesto nad Váhom,48.7590904055,17.8293864585
3,39507_22001949_P8948301,0,4000,24,DRG,8p133_20220316_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",IV,43,Program intenzívnej starostlivosti v neonatológii,p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22001949,5016945,NaN,522279,522279,"Detská fakultná nemocnica Košice, všeobecná ne...",803,48.721161,21.238323,Michalovce,807,Michalovce,48.7542280186,21.9107475822
4,39507_22002222_P8948301,0,1100,71,DRG,8p107_20220324_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",IV,43,Program intenzívnej starostlivosti v neonatológii,p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22002222,2668229,NaN,522872,522872,"Detská fakultná nemocnica Košice, všeobecná ne...",803,48.721161,21.238323,Pavlovce nad Uhom,807,Michalovce,48.7542280186,21.9107475822


In [70]:
df_hospitalizations_residency.shape

(1013114, 37)

In [71]:
df_hospitalizations_residency['ID_HP_ZP'].nunique()

1005472

In [72]:
df_hospitalizations_residency[df_hospitalizations_residency['id_okres_pacient'].str.lower().isin(['nan','n/a','missing', 'na',' ',''])].shape

(0, 37)

In [73]:
df_hospitalizations_residency['id_okres_pacient'].isna().sum()

13093

experimental branch

### Exploring duplicates

In [74]:
# Create a subset with rows where 'ID_HP_ZP' is duplicated
duplicates_subset = df_hospitalizations_residency[
    df_hospitalizations_residency.duplicated(subset='ID_HP_ZP', keep=False)
]

In [75]:
duplicates_subset.head(6)

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,ms_uroven,hlavny_program_cislo,hlavny_program_nazov,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY,NAZOV_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs,Obec_pacient_nazov,id_okres_pacient,Okres_pacient_nazov,lat_okres_pacient,lng_okres_pacient
9,1770_22003721_N4981301,0,3670,33,DRG,8p133_20220727_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",IV,43,Program intenzívnej starostlivosti v neonatológii,p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,N4981301,1770_22003721,2778925,NaN,522791,522791,Detská fakultná nemocnica s poliklinikou Bansk...,601,48.743041,19.121087,Moravany,807,Michalovce,48.7542280186,21.9107475822
10,1770_22003721_N4981301,0,3670,33,DRG,8p133_20220727_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",IV,43,Program intenzívnej starostlivosti v neonatológii,p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,N4981301,1770_22003721,2778925,NaN,522791,522791,Detská fakultná nemocnica s poliklinikou Bansk...,601,48.743041,19.121087,Moravany,807,Michalovce,48.7542280186,21.9107475822
247,20224300125027_P8948301,0,3360,-1,DRG,5h30210_20221125_Z,.*,43-01,Novorodenec so signifikantným OP výkonom,IV,43,Program intenzívnej starostlivosti v neonatológii,p06c,.*,.*,.*,drg,10,16,0,1,age0,P8948301,20224300125027,2587701001.0,NaN,527840,527840,"Detská fakultná nemocnica Košice, všeobecná ne...",803,48.721161,21.238323,Stropkov,711,Stropkov,49.2031385516,21.651376564
248,20224300125027_P8948301,0,3360,-1,DRG,5h30210_20221125_Z,.*,43-01,Novorodenec so signifikantným OP výkonom,IV,43,Program intenzívnej starostlivosti v neonatológii,p06c,.*,.*,.*,drg,10,16,0,1,age0,P8948301,20224300125027,2587701001.0,NaN,527840,527840,"Detská fakultná nemocnica Košice, všeobecná ne...",803,48.721161,21.238323,Stropkov,711,Stropkov,49.2031385516,21.651376564
282,8873_22009327_P4305901,0,3200,110,DRG,8q902_20220531_Z,8q902,43-06,Riadená hypotermia,IV,43,Program intenzívnej starostlivosti v neonatológii,p,.*,.*,.*,"drg, vykon",10,2,0,0,age0,P4305901,8873_22009327,5554963,NaN,513016,513016,"Národný ústav detských chorôb, špecializovaná ...",103,48.167618,17.090078,Dubnica nad Vahom,302,Ilava,48.9959360433,18.2337820216
283,8873_22009327_P4305901,0,3200,110,DRG,8q902_20220531_Z,8q902,43-06,Riadená hypotermia,IV,43,Program intenzívnej starostlivosti v neonatológii,p,.*,.*,.*,"drg, vykon",10,2,0,0,age0,P4305901,8873_22009327,5554963,NaN,513016,513016,"Národný ústav detských chorôb, špecializovaná ...",103,48.167618,17.090078,Dubnica nad Vahom,302,Ilava,48.9959360433,18.2337820216


In [76]:
duplicates_subset.shape

(14613, 37)

In [77]:
#duplicates_subset.to_csv("duplicates_subset_v4_0.csv")

### Getting rid of duplicates

see comment for v4.1

In [78]:
df_hospitalizations_residency = df_hospitalizations_residency.drop_duplicates(subset='ID_HP_ZP', keep='first')

In [79]:
df_hospitalizations_residency.shape

(1005472, 37)

In [80]:
df_hospitalizations_residency['id_okres_pacient'].isna().sum()

13034

### Final basic output

In [81]:
df_hospitalizations_residency.to_csv("df_hospitalizations_residency_v4_3.csv")

## Special outputs for PowerBI purposes (map)

### View by hospitals: Grouping patients by PZS and district

In [82]:
patient_distribution = df_hospitalizations_residency.groupby(['kod_nem', 'NAZOV_PZS', 'id_okres_pacient']).agg({
    'Okres_pacient_nazov': 'first',  # Assuming the name doesn't change within groups
    'lat_okres_pacient': 'first',  # Assuming each hospital has a unique set of coordinates
    'lng_okres_pacient': 'first',
    'ID_HP_ZP': 'size'  # If 'patient_id' uniquely identifies patients
}).reset_index().rename(columns={'ID_HP_ZP': 'patient_count_hospital_by_patient_district'})


In [83]:
patient_distribution

,kod_nem,NAZOV_PZS,id_okres_pacient,Okres_pacient_nazov,lat_okres_pacient,lng_okres_pacient,patient_count_hospital_by_patient_district
0,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",101,Bratislava I-V,48.1479920465,17.1009521666,2
1,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",205,Senica,48.6796340943,17.367240486,1
2,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",206,Skalica,48.8445967439,17.2249286367,1
3,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",207,Trnava,48.3736687196,17.5943146921,1
4,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",301,Bánovce nad Bebravou,48.7203256346,18.2523422892,2
...,...,...,...,...,...,...,...
13592,P9979505,"WESPA, s.r.o., liečebňa, Želiezovce",502,Čadca,49.442063437,18.7878733975,1
13593,P9979505,"WESPA, s.r.o., liečebňa, Želiezovce",511,Žilina,49.2191611238,18.7318907933,1
13594,P9979505,"WESPA, s.r.o., liečebňa, Želiezovce",606,Lučenec,48.3283882135,19.6689648273,1
13595,P9979505,"WESPA, s.r.o., liečebňa, Želiezovce",701,Bardejov,49.2956503658,21.2802594595,1


In [84]:
# Sort the results in descending order by patient count
patient_distribution_sorted = patient_distribution.sort_values(by=[ 'patient_count_hospital_by_patient_district'], ascending=[ False])

In [85]:
patient_distribution_sorted

,kod_nem,NAZOV_PZS,id_okres_pacient,Okres_pacient_nazov,lat_okres_pacient,lng_okres_pacient,patient_count_hospital_by_patient_district
799,N3306701,Fakultná nemocnica s poliklinikou J. A. Reiman...,707,Prešov,48.9936856017,21.2434825675,21575
2618,N9272501,"Fakultná nemocnica s poliklinikou Žilina, všeo...",511,Žilina,49.2191611238,18.7318907933,20451
10482,P7701701,"Univerzitná nemocnica L.Pasteura Košice, všeob...",802,Košice I-IV,48.7251617291,21.2554474655,19720
12067,P8568701,"Fakultná nemocnica Nitra, všeobecná nemocnica",403,Nitra,48.3091486533,18.079527695,16294
6263,P4070701,Univerzitná nemocnica Bratislava,101,Bratislava I-V,48.1479920465,17.1009521666,15242
...,...,...,...,...,...,...,...
5680,P3660501,"Nemocnica s poliklinikou Spišská Nová Ves, a.s...",301,Bánovce nad Bebravou,48.7203256346,18.2523422892,1
5675,P3660501,"Nemocnica s poliklinikou Spišská Nová Ves, a.s...",203,Hlohovec,48.4289014119,17.7970101487,1
5667,P3596801,Stredoslovenský ústav srdcových a cievnych cho...,Unknown,Unknown,48.6165040594157,16.9401676189405,1
5666,P3596801,Stredoslovenský ústav srdcových a cievnych cho...,UA,UA,48.5420509368223,22.3764011118678,1


In [86]:
patient_distribution_sorted.to_csv("patient_distribution_sorted.csv")

### Extended view by hospitals: Grouping patients by PZS, district, Program name and ms_uroven

In [87]:
patient_distribution_detailed = df_hospitalizations_residency.groupby(['kod_nem', 'NAZOV_PZS', 'id_okres_pacient', 'hlavny_program_nazov', 'ms_uroven']).agg({
    'Okres_pacient_nazov': 'first',  # Assuming the name doesn't change within groups
    'lat_okres_pacient': 'first',  # Assuming each hospital has a unique set of coordinates
    'lng_okres_pacient': 'first',
    'ID_HP_ZP': 'size'  # If 'patient_id' uniquely identifies patients
}).reset_index().rename(columns={'ID_HP_ZP': 'patient_count_hospital_by_patient_district_program_mslevel'})

In [88]:
patient_distribution_detailed.head()

,kod_nem,NAZOV_PZS,id_okres_pacient,hlavny_program_nazov,ms_uroven,Okres_pacient_nazov,lat_okres_pacient,lng_okres_pacient,patient_count_hospital_by_patient_district_program_mslevel
0,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",101,Neurologický program,II,Bratislava I-V,48.1479920465,17.1009521666,1
1,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",101,Program klinickej onkológie,II,Bratislava I-V,48.1479920465,17.1009521666,1
2,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",205,Neurologický program,II,Senica,48.6796340943,17.367240486,1
3,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",206,Neinvazívny kardiovaskulárny program,II,Skalica,48.8445967439,17.2249286367,1
4,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",207,"Program endokrinológie, diabetológie a metabol...",II,Trnava,48.3736687196,17.5943146921,1


In [89]:
patient_distribution_detailed_sorted = patient_distribution_detailed.sort_values(by=[ 'patient_count_hospital_by_patient_district_program_mslevel'], ascending=[ False])

In [90]:
patient_distribution_detailed_sorted.head()

,kod_nem,NAZOV_PZS,id_okres_pacient,hlavny_program_nazov,ms_uroven,Okres_pacient_nazov,lat_okres_pacient,lng_okres_pacient,patient_count_hospital_by_patient_district_program_mslevel
33494,P4070701,Univerzitná nemocnica Bratislava,101,Oftalmologický program,I,Bratislava I-V,48.1479920465,17.1009521666,3461
37017,P4070703,Univerzitná nemocnica Bratislava,101,Oftalmologický program,I,Bratislava I-V,48.1479920465,17.1009521666,3036
2184,N2200101,"Nemocnica Poprad, a. s., všeobecná nemocnica",706,Oftalmologický program,I,Poprad,49.0543940271,20.2936001048,2736
69843,P8568701,"Fakultná nemocnica Nitra, všeobecná nemocnica",403,Oftalmologický program,I,Nitra,48.3091486533,18.079527695,2470
15683,N9272501,"Fakultná nemocnica s poliklinikou Žilina, všeo...",511,Oftalmologický program,I,Žilina,49.2191611238,18.7318907933,2394


In [91]:
patient_distribution_detailed_sorted.shape

(77578, 9)

In [92]:
patient_distribution_detailed_sorted.to_csv("patient_distribution_detailed_sorted.csv")

### More extended view for mSMA: Grouping patients by PZS, district, Program name, ms_uroven and ms

In [93]:
patient_distribution_detailed_withMS = df_hospitalizations_residency.groupby(['kod_nem', 'NAZOV_PZS', 'id_okres_pacient', 'hlavny_program_nazov', 'ms_uroven', 'kod_ms', 'ms_name']).agg({
    'Okres_pacient_nazov': 'first',  # Assuming the name doesn't change within groups
    'lat_okres_pacient': 'first',  # Assuming each hospital has a unique set of coordinates
    'lng_okres_pacient': 'first',
    'ID_HP_ZP': 'size'  # If 'patient_id' uniquely identifies patients
}).reset_index().rename(columns={'ID_HP_ZP': 'patient_count_hospital_by_patient_district_program_mslevel_concreteMS'})

In [95]:
patient_distribution_detailed_withMS_sorted = patient_distribution_detailed_withMS.sort_values(by=[ 'patient_count_hospital_by_patient_district_program_mslevel_concreteMS'], ascending=[ False])

In [102]:
patient_distribution_detailed_withMS_sorted['ms_kod_nazov'] = patient_distribution_detailed_withMS_sorted['kod_ms'] + ' ' + patient_distribution_detailed_withMS_sorted['ms_name']

In [103]:
patient_distribution_detailed_withMS_sorted.head()

,kod_nem,NAZOV_PZS,id_okres_pacient,hlavny_program_nazov,ms_uroven,kod_ms,ms_name,Okres_pacient_nazov,lat_okres_pacient,lng_okres_pacient,patient_count_hospital_by_patient_district_program_mslevel_concreteMS,ms_kod_nazov
65424,P4070701,Univerzitná nemocnica Bratislava,101,Oftalmologický program,I,05-43,Terapeutické podanie látok do oka,Bratislava I-V,48.1479920465,17.1009521666,2363,05-43 Terapeutické podanie látok do oka
72814,P4070703,Univerzitná nemocnica Bratislava,101,Oftalmologický program,I,05-43,Terapeutické podanie látok do oka,Bratislava I-V,48.1479920465,17.1009521666,2314,05-43 Terapeutické podanie látok do oka
15801,N4223101,Fakultná nemocnica s poliklinikou F.D.Roosevel...,601,Oftalmologický program,I,05-43,Terapeutické podanie látok do oka,Banská Bystrica,48.7304254695,19.1384766527,1636,05-43 Terapeutické podanie látok do oka
32075,N9272501,"Fakultná nemocnica s poliklinikou Žilina, všeo...",511,Oftalmologický program,I,05-43,Terapeutické podanie látok do oka,Žilina,49.2191611238,18.7318907933,1626,05-43 Terapeutické podanie látok do oka
62030,P3881101,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,Oftalmologický program,I,05-43,Terapeutické podanie látok do oka,Martin,49.0754269836,18.9165750198,1479,05-43 Terapeutické podanie látok do oka


In [104]:
patient_distribution_detailed_withMS_sorted.shape

(153660, 12)

In [105]:
patient_distribution_detailed_withMS_sorted.to_csv("patient_distribution_detailed_withMS_sorted.csv")

### Hospitals and their coordinates

In [99]:
hospital_coordinates_patientcounts = df_hospitalizations_residency.groupby(['kod_nem', 'NAZOV_PZS',]).agg({
    'lat_pzs': 'first',  # Assuming each hospital has a unique set of coordinates
    'lng_pzs': 'first',
    'ID_HP_ZP': 'size'  # If 'patient_id' uniquely identifies patients
}).reset_index().rename(columns={'ID_HP_ZP': 'patient_count_hospital'})


In [100]:
hospital_coordinates_patientcounts

,kod_nem,NAZOV_PZS,lat_pzs,lng_pzs,patient_count_hospital
0,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",49.1218172755,20.446369775,94
1,N1949101,"Sanatórium Tatranská Kotlina, n.o., špecializo...",49.1445802074,20.2442966301,799
2,N1968101,Národný ústav detskej tuberkulózy a respiračný...,49.132872,20.238762,2391
3,N1969701,"Sanatórium Dr. Guhra, n.o., špecializovaná nem...",49.1445802074,20.2442966301,626
4,N2114901,"Kysucká nemocnica s poliklinikou Čadca, všeobe...",49.439411,18.772795,12328
...,...,...,...,...,...
246,P9331901,"GYN - FIV a.s., špecializovaná ambulancia, Bra...",48.1501716944,17.1606706666,108
247,P9332901,Špecializovaná nemocnica pre ortopedickú prote...,48.154241,17.129727,1184
248,P9755401,"Nemocnica arm. generála L. Svobodu Svidník, a....",49.311812,21.572807,5130
249,P9932701,"GYN – FIV Žilina, s. r. o., zariadenie jednodň...",49.2191611238,18.7318907933,212


In [101]:
hospital_coordinates_patientcounts.to_csv("hospital_coordinates_patientcounts.csv")

# Data exploration